In [1]:
from keras.models import load_model
from keras.preprocessing.sequence import pad_sequences

In [2]:
model = load_model('model20171223.h5')

ResourceExhaustedError: OOM when allocating tensor with shape[8762]
	 [[Node: dense_6/bias/Assign = Assign[T=DT_FLOAT, _class=["loc:@dense_6/bias"], use_locking=true, validate_shape=true, _device="/job:localhost/replica:0/task:0/device:GPU:0"](dense_6/bias, dense_6/Const)]]

Caused by op 'dense_6/bias/Assign', defined at:
  File "/home/fei/miniconda3/envs/py3_5/lib/python3.5/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/fei/miniconda3/envs/py3_5/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/fei/miniconda3/envs/py3_5/lib/python3.5/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/fei/miniconda3/envs/py3_5/lib/python3.5/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/home/fei/miniconda3/envs/py3_5/lib/python3.5/site-packages/ipykernel/kernelapp.py", line 478, in start
    self.io_loop.start()
  File "/h

In [ ]:
def load_dict(filename):
    with open(filename, 'r') as f:
        return json.load(f)

def load_index_word_map(word2ind_filename='word2ind', ind2word_filename='ind2word'):
    word2ind = load_dict(word2ind_filename)
    ind2word = load_dict(ind2word_filename)
    return word2ind, ind2word

In [3]:
pad_sequences([[1, 2]], maxlen=10, value=0)

array([[0, 0, 0, 0, 0, 0, 0, 0, 1, 2]], dtype=int32)